# One-vs-All Classification

## Table of Contents
- [Introduction](#intro)
- [The Dataset](#dataset)
- [The Regularized Cost Function and Gradient](#cost_and_gradient)
- [Training the Models](#training)
- [Making Predictions](#predictions)

<a id='intro'></a>
### Introduction
In a [previous article](https://github.com/marty-vanhoof/Maching_Learning/blob/master/Logistic_Regression/Logistic_Regression.ipynb), we talked in detail about logistic regression in the case of binary classification; that is, given a set of training data $\{ \, ( \mathbf{x}_1,y_1 ), ( \mathbf{x}_2,y_2 ) \ldots,  ( \mathbf{x}_m,y_m ) \, \}$, logistic regression finds the parameters $\theta_0, \theta_1, \ldots, \theta_n$ of a function $h_\theta(\mathbf{x})$ that estimates the probability of a class label $y=1$ given a feature vector $\mathbf{x}$, and so the function $1 - h_\theta(\mathbf{x})$ estimates the probability of a class label $y=0$ given $\mathbf{x}$.  We can then predict binary outcomes by specifying a probability threshold, and the threshold we use is given as follows

$$ \mathrm{prediction} =
\begin{cases}
1 & \mathrm{if} \quad P(y = 1 \,|\, \mathbf{x} ; \theta) \geq 0.5 \\ 
0 & \mathrm{if} \quad P(y = 0 \,|\, \mathbf{x} ; \theta) < 0.5
\end{cases} 
$$

One-vs-all classification extends logistic regression to the case of classifying more than two class labels by considering a sequence of binary classifications.  Suppose the target variable $y$ can be labeled in $k$ possible ways, so $y \in \{1, 2, \ldots, k \}$ with $k \geq 2$.  We can turn this into a sequence of binary classification problems and use logistic regression at each step.  

First separate the labels $\{1, 2, \dots, k \}$ into either 1 or $\{2, \dots, k \}$ and fit a standard logistic regression model to this.  This logistic regression model will find a function $h_\theta^{(1)}(\mathbf{x})$ that classifies the data into either 1 or $\{ 2, \ldots, k \}$.

Next, separate the labels into either 2 or $\{ 1, 3, \dots, k \}$ and fit another logistic regression model to this, resulting in a function $h_\theta^{(2)}(\mathbf{x})$ that classifies the data into either 2 or $\{ 1, 3, \dots, k \}$.
And so on...the last step will fit another logistic regression model that finds a function $h_\theta^{(k)}(\mathbf{x})$ that classifies the data into either $k$ or $\{ 1, 2, \ldots, k-1 \}$.

The result is that we have $k$ classifiers $h_\theta^{(1)}(\mathbf{x}), h_\theta^{(2)}(\mathbf{x}), \ldots, h_\theta^{(k)}(\mathbf{x})$ each trying to estimate the probability of a class label $y \in \{ 1, 2, \ldots, k \}$ given $\mathbf{x}$.  That is,

$$
h_\theta^{(i)}(\mathbf{x}) = P(y = i \,|\, \mathbf{x}; \theta) \,, \quad i = 1,2, \ldots, k
$$

Then to make a prediction on a feature vector $\mathbf{x}$, pick the class $i$ that gives maximum probability

$$
\underset{i}{\mathrm{argmax}} \, h_\theta^{(i)}(\mathbf{x}) 
$$

A nice discussion of one-vs-all classification can be found in [this video](https://www.youtube.com/watch?v=BzSsQWhDRXE&list=PLZ9qNFMHZ-A4rycgrgOYma6zxF4BZGGPW).

<a id='dataset'></a>
### The Dataset
As usual, we will be working through an example from one of the programming exercises in Andrew Ng's machine learning course.  We will implement one-vs-all logistic regression to recognize hand-written digits.  The [dataset](https://github.com/marty-vanhoof/Maching_Learning/blob/master/data/logReg_data3.mat) contains 5000 training examples of handwritten digits, which is a subset of the MNIST database of handwritten digits available [here](http://yann.lecun.com/exdb/mnist/).  This picture below is an example of some of the digits.
<br/>
<img src="digits.png" height="400" width="400">
<br/>
Each training example (each little box) is a 20 pixel by 20 pixel grayscale image of the digit.  The image can be represented as a 400 dimensional vector $\mathbf{x}_i$ of floating-point numbers, where each number represents the grayscale intensity of a pixel at that location.  Each of these training examples becomes a single row in a feature matrix $X$.  The class labels are in the target vector $\mathbf{y} = (y_1, y_2, \ldots, y_{400})^T$, where each $y_i$ can be one of the numbers 0 through 9

$$
X =
\begin{bmatrix}
\, - \,\, \mathbf{x}_1^T - \, \\
\, - \,\, \mathbf{x}_2^T - \, \\
\vdots \\
\, - \,\, \mathbf{x}_{400}^T - \,
\end{bmatrix} \, , \quad
\mathbf{y} =
\begin{bmatrix}
y_1 \\ y_2 \\ \vdots \\ y_{400} 
\end{bmatrix} 
$$ 

The dataset is in a MATLAB format, so we have to use a function `loadmat()` in the SciPy library in order for Pandas to interpret it.

In [1]:
import numpy as np  
import pandas as pd 
import os
import matplotlib.pyplot as plt  
from scipy.io import loadmat  
%matplotlib inline

filepath = os.getcwd() + '/logReg_data3.mat'
data = loadmat('logReg_data3.mat')
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

.

We should make sure that $X$ and $\mathbf{y}$ have the proper shapes.

In [2]:
#X, y = data['X'], data['y']
print(data['X'].shape, data['y'].shape)

(5000, 400) (5000, 1)


<a id='cost_and_gradient'></a>
### The Regularized Cost Function and Gradient

Since we are going to use one-vs-all logistic regression to classify the digits 0 through 9, we will need to train 10 separate logistic regression models.  We will re-use the logistic cost function and the gradient code from before, and we will also implement the regularized versions of these functions.  In matrix (vectorized) form, the regularized cost function is given as

$$ J(\theta) = -\frac{1}{m} \big[\, \mathbf{y}^T \mathbf{ln}_h + (\mathbf{1} - \mathbf{y})^T \mathbf{ln}_{1 - h} \,\big] + \frac{\lambda}{2m} ||\, \widehat{\theta} \,||^2 \,, $$

where

$$
\mathbf{1} =
\begin{bmatrix}
1 \\
1 \\
\vdots \\
1
\end{bmatrix} \, , \quad
\mathbf{ln}_h =
\begin{bmatrix}
\ln h_\theta(\mathbf{x}_1) \\
\ln h_\theta(\mathbf{x}_2) \\
\vdots \\ 
\ln h_\theta(\mathbf{x}_m)
\end{bmatrix} \, , \quad
\mathbf{ln}_{1-h} =
\begin{bmatrix}
\ln (1 - h_\theta(\mathbf{x}_1)) \\
\ln (1 - h_\theta(\mathbf{x}_2)) \\
\vdots \\ 
\ln (1 - h_\theta(\mathbf{x}_m)) 
\end{bmatrix} \, , \quad
\widehat{\theta} =
\begin{bmatrix}
\theta_1 \\ \theta_2 \\ \vdots \\ \theta_n
\end{bmatrix} \,,
$$


and $m$ is the number of training examples.  The Python function `regularized_logReg_cost()` implements $J(\theta)$ below.

The gradient of the regularized cost function is the vector $\nabla J(\theta)$ whose components are given by

\begin{align*}
\frac{\partial J}{\partial \theta_0} &= \frac{1}{m} \sum_{i=1}^m (h_\theta(\mathbf{x}_i) - y_i) x_{ij} \quad \textrm{for } j=0 \\
\frac{\partial J}{\partial \theta_j} &= \frac{1}{m} \bigg( \sum_{i=1}^m (h_\theta(\mathbf{x}_i) - y_i) x_{ij} + \lambda \theta_j \bigg) \quad \textrm{for } j \geq 1
\end{align*}

The computation of the gradient $\nabla J(\theta)$ is implemented in the function `regularized_logReg_gradient()` below in vectorized form.

In [3]:
from log_reg_scripts import logReg_cost, logistic

def regularized_logReg_cost(theta, X, y, lambda_):
    '''Compute the regularized cost function.  This is almost the same as
    logReg_cost() from before, but now we're adding a regularization term.
    '''
    # re-use logReg_cost from before
    unreg_cost = logReg_cost(theta, X, y)
    # convert theta back to a numpy array since logReg_cost converts it to a numpy matrix
    theta = np.array(theta)
    reg_term = lambda_ / (2*len(y)) * sum(theta[1:]**2) 
    return unreg_cost + reg_term

def regularized_logReg_gradient(theta, X, y, lambda_):
    '''Compute the gradient of the regularized logistic cost function.
    '''
    
    theta, X, y = np.matrix(theta), np.matrix(X), np.matrix(y)
    
    h_theta = logistic(X*theta.T)
    gradient = ( 1 / len(y) )*(h_theta - y).T * X 
    theta_temp = theta
    theta_temp[0, 0] = 0
    reg_gradient = gradient + (lambda_ / len(y))*theta_temp
    
    return np.array(reg_gradient)


<a id='training'></a>
### Training the Models

We are now going to train the 10 different classifiers using the function `one_vs_all()` below.  Standard logistic regression can only distinguish between two classes at a time, so given our labels $L = \{0,1,2, \ldots,9 \}$, each logistic regression model will classify the data into "class $i$" or "not class $i$", where $i$ is a number in our labels set $L$. 

The function below will loop over each class label, and then find the optimal theta parameters for each label using the [minimize](https://docs.scipy.org/doc/scipy/reference/optimize.html) function in SciPy's optimize module.

In [4]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, lambda_):
    '''
    Trains num_labels logisitic regression classifiers and returns each of these classifiers
    in a matrix all_theta, where the i-th row of all_theta corresponds to the classifier 
    for label i
    '''
    
    # m = number of training examples, n = number of features
    m, n = X.shape[0], X.shape[1]
    
    # all_theta will be populated with all the parameters of our model
    all_theta = np.zeros((num_labels, n+1))
    
    # insert column of ones into X
    X = np.insert(X, 0, np.ones(m), axis=1)
    
    # for each label, train a separate classifier
    # Note: the labels are 1-indexed instead of 0-indexed due MATLAB being 1-indexed
    for i in range(1, num_labels+1):
        theta = np.zeros(n+1)
        y_i = (y == i).astype(int)
        # use scipy.optimize's minimize function to minimize regularized_logReg_cost
        # and store the resulting parameters in all_theta
        result = minimize(fun=regularized_logReg_cost, x0=theta, args=(X, y_i, lambda_),
                        method='TNC', jac=regularized_logReg_gradient)
        all_theta[i-1, :] = result.x
        
    return all_theta

all_theta = one_vs_all(data['X'], data['y'], 10, 1)    

<a id='predictions'></a>
### Making Predictions

Now we are ready to use our model to make predictions and then evaluate the accuracy on the training set.  Recall that the **accuracy** is defined as the number of correct predictions divided by the total number of predictions.  Of course, evaluating the accuracy only on the training set is a big mistake in machine learning because the ultimate goal is that we want the model to generalize to new unseen data.  The importance of splitting the data into training and testing sets will be discussed in a future write-up.

Recall from our discussion at the beginning that after training all of the classifiers, we can make a prediction on a feature vector $\mathbf{x}$ by picking the class that gives maximum probability

$$
 \underset{i}{\mathrm{argmax}} \, P(y = i \,|\, \mathbf{x}; \theta) \,.
$$

The `predict()` function implements this below and then the `accuracy()` function computes the accuracy of the model on the training data.  We can see that on the training set, the percentage of correct predictions is 94.46%.  This is nice, but we have no idea if the model is overfitting since we didn't compare the accuracy on a test set.

In the next article we will see how to use a neural network to classify handwritten digits using the same training set we used here.

In [5]:
def predict(all_theta, X):
    '''
    Predict the label for each training example in X. Returns an array of m predictions.
    
    all_theta is a matrix (n-dim numpy array) where the i-th row is a trained logistic
    regression theta vector for the i-th class.
    X is the feature matrix with m rows (each row being a training example).
    '''
    m = X.shape[0]
    num_labels = all_theta.shape[0]
    # insert column of ones into X
    X = np.insert(X, 0, np.ones(m), axis=1)
    # probs stores num_labels class probabilities for each training example
    probs = logistic(X @ all_theta.T)
    # pick the label with max probability for each training example
    # Note: the labels are 1-indexed instead of 0-indexed due MATLAB being 1-indexed
    predictions = np.argmax(probs, axis = 1) + 1
    
    return predictions

def accuracy(predictions, y):
    '''
    Compute the accuracy of the model by comparing the
    array of predictions to the target vector y.
    Returns the proportion of correct predictions.
    '''
    y = y.reshape((5000,))
    correct = (predictions == y).astype(int)
    accuracy = np.sum(correct) / len(predictions)
    
    return accuracy

In [6]:
predictions = predict(all_theta, data['X'])
accuracy_percent = accuracy(predictions, data['y']) * 100
print('accuracy: {}%'.format(accuracy_percent))

accuracy: 94.46%
